# Proyecto 1 - Mineria de datos
## Explorando patrones de robos por edad de las victimas en Guatemala

### Autores:
#### Andrés de la Roca
#### Jun Woo Lee

### Analisis exploratorio

In [ ]:
print("Hi Mom!")

### Analisis por clustering